In [2]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.Collecting pyspark
     ---------------------------------------- 0.0/317.3 MB ? eta -:--:--
     --------------------------------------- 0.3/317.3 MB 10.2 MB/s eta 0:00:31
     --------------------------------------- 1.6/317.3 MB 17.4 MB/s eta 0:00:19
     --------------------------------------- 3.6/317.3 MB 28.5 MB/s eta 0:00:12
      -------------------------------------- 5.3/317.3 MB 30.6 MB/s eta 0:00:11
      -------------------------------------- 6.5/317.3 MB 32.1 MB/s eta 0:00:10
     - ------------------------------------- 8.6/317.3 MB 32.6 MB/s eta 0:00:10
     - ------------------------------------ 10.3/317.3 MB 34.4 MB/s eta 0:00:09
     - ------------------------------------ 11.8/317.3 MB 38.5 MB/s eta 0:00:08
     - ------------------------------------ 13.9/317.3 MB 38.5 MB/s eta 0:00:08
     - ------------------------------------ 15.8/317.3 MB 38.5 MB/s eta 0:00:08
     -- -----------------------------------

  DEPRECATION: pyspark is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  error: subprocess-exited-with-error
  
  × Running setup.py install for pyspark did not run successfully.
  │ exit code: 1
  ╰─> [2132 lines of output]
      C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\site-packages\setuptools\dist.py:771: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
        warnings.warn(
      running install
      C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\site-packages\setuptools\command\install.py:34

# RAW: STANDINGS API NHL

In [1]:
from pyspark.sql import SparkSession
import requests
import pandas

# Start Spark session
spark = SparkSession.builder.appName("NHL Standings").getOrCreate()

# Make the API request for standings
response = requests.get("https://api-web.nhle.com/v1/standings/now")
standings_data = response.json()  # Correct method to parse JSON from the response

# Create a DataFrame directly from the JSON data
df = spark.createDataFrame([standings_data])
# Save the DataFrame as a JSON file
#df.write.mode("overwrite").json("Files/NHL_RAW/standings_now.json")
display(df)


PySparkRuntimeError: [JAVA_GATEWAY_EXITED] Java gateway process exited before sending its port number.

# #RAW: TEAM DATA API STORAGE CALL

In [3]:
teams = requests.get("https://api.nhle.com/stats/rest/en/team")
teams_data = teams.json()  # Correct method to parse JSON from the response

# Create a DataFrame directly from the JSON data for storing data related to the teams
df = spark.createDataFrame([teams_data])
# Save the DataFrame as a JSON file
df.write.mode("overwrite").json("Files/NHL_RAW/teams.json")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 4, Finished, Available, Finished)

### Standings Needs some adjustment still

In [5]:
from pyspark.sql.functions import col, explode, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Assuming 'df' is your DataFrame and 'standings' is the column name containing the JSON string

# Define the schema of your JSON data as it appears in 'standings' if known; modify as needed

# Complete JSON schema based on the provided structure
schema = StructType([
    StructField("ties", StringType()),
    StructField("conferenceL10Sequence", StringType()),
    StructField("homeGoals", StringType()),
    StructField("homeGoalsAgainst", StringType()),
    StructField("homeRegulationPlusOtWins", StringType()),
    StructField("divisionAbbrev", StringType()),
    StructField("homeGoalsFor", StringType()),
    StructField("shootoutLosses", StringType()),
    StructField("otLosses", StringType()),
    StructField("leagueL10Sequence", StringType()),
    StructField("goalDifferential", StringType()),
    StructField("homeLosses", StringType()),
    StructField("homePoints", StringType()),
    StructField("teamName", StringType()),
    StructField("streakCode", StringType()),
    StructField("regulationWins", StringType()),
    StructField("homeRegulationWins", StringType()),
    StructField("pointPct", StringType())
])
df = spark.read.option("multiline", "true").json("Files/NHL_RAW/standings_now.json")
# df now is a Spark DataFrame containing JSON data from "Files/NHL_RAW/standings_now.json".
display(df)
# Assuming 'df' is your DataFrame and 'standings' is the column name containing the JSON string
df = spark.read.schema(schema).json(df.select("standings").rdd.map(lambda x: x.standings))


# Convert JSON string in 'standings' to actual structured data
df = df.withColumn("json_data", from_json(col("standings"), schema))

# Explode the array to create a new row for each element in the JSON array
df = df.withColumn("exploded_data", explode(col("json_data")))

# Select the fields you want to flatten, you may need to adjust the path based on actual JSON structure
df = df.select(
    col("exploded_data.ties").alias("ties"),
    col("exploded_data.conferenceL10Sequence").alias("conferenceL10Sequence"),
    col("exploded_data.homeGoals").alias("homeGoals"),
    col("exploded_data.homeGoalsAgainst").alias("homeGoalsAgainst"),
    col("exploded_data.homeRegulationPlusOtWins").alias("homeRegulationPlusOtWins"),
    col("exploded_data.divisionAbbrev").alias("divisionAbbrev"),
    col("exploded_data.homeGoalsFor").alias("homeGoalsFor"),
    col("exploded_data.shootoutLosses").alias("shootoutLosses"),
    col("exploded_data.otLosses").alias("otLosses"),
    col("exploded_data.leagueL10Sequence").alias("leagueL10Sequence"),
    col("exploded_data.goalDifferential").alias("goalDifferential"),
    col("exploded_data.homeLosses").alias("homeLosses"),
    col("exploded_data.homePoints").alias("homePoints"),
    col("exploded_data.teamName").alias("teamName"),
    col("exploded_data.streakCode").alias("streakCode"),
    col("exploded_data.regulationWins").alias("regulationWins"),
    col("exploded_data.homeRegulationWins").alias("homeRegulationWins"),
    col("exploded_data.pointPct").alias("pointPct")
)

# Display the flattened data
df.show()



StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 6, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 10f0a36f-ba6f-4806-ae02-e39559d5756f)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `standings` cannot be resolved. Did you mean one of the following? [`ties`, `teamName`, `homePoints`, `otLosses`, `pointPct`].;
'Project [ties#806, conferenceL10Sequence#807, homeGoals#808, homeGoalsAgainst#809, homeRegulationPlusOtWins#810, divisionAbbrev#811, homeGoalsFor#812, shootoutLosses#813, otLosses#814, leagueL10Sequence#815, goalDifferential#816, homeLosses#817, homePoints#818, teamName#819, streakCode#820, regulationWins#821, homeRegulationWins#822, pointPct#823, from_json(StructField(ties,StringType,true), StructField(conferenceL10Sequence,StringType,true), StructField(homeGoals,StringType,true), StructField(homeGoalsAgainst,StringType,true), StructField(homeRegulationPlusOtWins,StringType,true), StructField(divisionAbbrev,StringType,true), StructField(homeGoalsFor,StringType,true), StructField(shootoutLosses,StringType,true), StructField(otLosses,StringType,true), StructField(le

## PREPARE TEAMS ETL WILL BE NEEDED LATER FOR A LOOP CALL

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode

# Assuming you're already in a PySpark Notebook in Fabric
# Load the JSON data into a DataFrame
df = spark.read.option("multiline", "true").json("Files/NHL_RAW/teams.json")

# Explode the array to create individual rows for each JSON object
df_exploded = df.withColumn("data", explode(col("data")))

# Select individual fields from the exploded data
df_final = df_exploded.select(
    col("data.franchiseId").alias("franchiseId"),
    col("data.id").alias("TeamId"),
    col("data.leagueId").alias("leagueId")
)

# Display the final dataframe
display(df_final)
df_final.write.mode("overwrite").csv("Files/NHL_ETL/TEAMS_ETL/teams.csv")
print(df_final)
unique_teams = df_final

StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, f8c6e32e-75e9-4616-aedc-26b433bb9283)

DataFrame[franchiseId: bigint, TeamId: bigint, leagueId: bigint]


# RAW: TEAM MATCHUP HISTORY

In [7]:
import requests
from pyspark.sql import SparkSession

# Assuming SparkSession has already been created
spark = SparkSession.builder.getOrCreate()

details_list = []  # Temporary list to store all responses

# Collect the TeamId column into a list
team_ids = [row.TeamId for row in df_final.select("TeamId").collect()]

# Iterating through all TeamIds
for teamid in team_ids:
    teamdetails = requests.get(f"https://records.nhl.com/site/api/all-time-record-vs-franchise?cayenneExp=teamFranchiseId={teamid}")
    
    if teamdetails.status_code == 200:  # Ensure the request was successful
        teams_data = teamdetails.json()  # Parse JSON from the response
        if "data" in teams_data:  # Assuming the actual data is nested under a key called 'data'
            details_list.extend(teams_data["data"])  # Append the 'data' to the list

# Creating a DataFrame from the collected list
details_df = spark.createDataFrame(details_list)

details_df = details_df.withColumnRenamed("franchiseName", "PriorityFranchiseName")

display(details_df)

# Optional: Saving the details DataFrame to a JSON file
details_df.write.mode("overwrite").option("header","true").csv("Files/NHL_RAW/FranchiseMatchupHistory.csv")
##If you come back to this in the future, rember to specify headers on write so that when you bring in for next transformation csv the data
#Display final result: 



StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c31c07f3-c3b9-42b0-944e-3fbd3218d57d)

In [8]:
df = spark.read.format("csv").option("header","true").load("Files/NHL_RAW/FranchiseMatchupHistory.csv")
# df now is a Spark DataFrame containing CSV data from "Files/NHL_RAW/FranchiseMatchupHistory.csv".
display(df)

StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, d9adba6a-202f-482c-98ee-37b215cd9403)

# RAW: Summary Data & KPIs

In [10]:
summary = requests.get("https://api.nhle.com/stats/rest/en/team/summary")
summary_data = summary.json()  # Correct method to parse JSON from the response


# Typically, APIs will nest the actual data you want within a key, assuming it's under 'data'
# It's common for APIs to wrap results in a structure like {"data": [actual_data]}
summary_data = summary_data.get('data', [])  # Replace 'data' with the correct key if different



# Create a DataFrame directly from the JSON data for storing data related to the teams
df = spark.createDataFrame([summary_data])
# Save the DataFrame as a JSON file
df.write.mode("overwrite").json("Files/NHL_RAW/teams_summary.json")

df = spark.read.option("multiline", "true").json("Files/NHL_RAW/teams_summary.json")
# df now is a Spark DataFrame containing JSON data from "Files/NHL_RAW/teams_summary.json".
display(summary_data)
display(df)

StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 11, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 561ba679-5118-4251-a815-30321e2fec10)

SynapseWidget(Synapse.DataFrame, bb106ce6-4ad3-4e09-8123-133f483ea224)

## NEEDS FIXED COMEBACK AND ADJUST BEFORE MOVING ON to PROD OR ETL FLOW

###

## ###ETL FOR TEEAMS SUMMARY

In [11]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.getOrCreate()

# Load the JSON file
raw_df = spark.read.option("multiline", "true").json("Files/NHL_RAW/teams_summary.json")

# Get all column names corresponding to the nested structures dynamically
nested_columns = [col for col in raw_df.columns if col.startswith("_")]

# Initialize an empty DataFrame to union all flattened structures
flattened_df = None

# Loop through each nested structure and flatten it
for nested_col in nested_columns:
    temp_df = raw_df.select(
        col(f"{nested_col}.faceoffWinPct").alias("faceoffWinPct"),
        col(f"{nested_col}.gamesPlayed").alias("gamesPlayed"),
        col(f"{nested_col}.goalsAgainst").alias("goalsAgainst"),
        col(f"{nested_col}.goalsAgainstPerGame").alias("goalsAgainstPerGame"),
        col(f"{nested_col}.goalsFor").alias("goalsFor"),
        col(f"{nested_col}.goalsForPerGame").alias("goalsForPerGame"),
        col(f"{nested_col}.losses").alias("losses"),
        col(f"{nested_col}.otLosses").alias("otLosses"),
        col(f"{nested_col}.penaltyKillNetPct").alias("penaltyKillNetPct"),
        col(f"{nested_col}.penaltyKillPct").alias("penaltyKillPct"),
        col(f"{nested_col}.pointPct").alias("pointPct"),
        col(f"{nested_col}.points").alias("points"),
        col(f"{nested_col}.powerPlayNetPct").alias("powerPlayNetPct"),
        col(f"{nested_col}.powerPlayPct").alias("powerPlayPct"),
        col(f"{nested_col}.regulationAndOtWins").alias("regulationAndOtWins"),
        col(f"{nested_col}.seasonId").alias("seasonId"),
        col(f"{nested_col}.shotsAgainstPerGame").alias("shotsAgainstPerGame"),
        col(f"{nested_col}.shotsForPerGame").alias("shotsForPerGame"),
        col(f"{nested_col}.teamFullName").alias("teamFullName"),
        col(f"{nested_col}.teamId").alias("teamId"),
        col(f"{nested_col}.ties").alias("ties"),
        col(f"{nested_col}.wins").alias("wins"),
        col(f"{nested_col}.winsInRegulation").alias("winsInRegulation"),
        col(f"{nested_col}.winsInShootout").alias("winsInShootout")
    )
    
    # Combine all structures into a single DataFrame
    flattened_df = temp_df if flattened_df is None else flattened_df.union(temp_df)

# Save the final flattened DataFrame as a CSV file
flattened_df.write.mode("overwrite").option("header", "true").csv("Files/NHL_ETL/STANDINGS_ETL/teams_summary.csv")

# Display the result for verification
flattened_df.show()


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 12, Finished, Available, Finished)

+-------------+-----------+------------+-------------------+--------+---------------+------+--------+-----------------+--------------+--------+------+---------------+------------+-------------------+--------+-------------------+---------------+------------+------+----+----+----------------+--------------+
|faceoffWinPct|gamesPlayed|goalsAgainst|goalsAgainstPerGame|goalsFor|goalsForPerGame|losses|otLosses|penaltyKillNetPct|penaltyKillPct|pointPct|points|powerPlayNetPct|powerPlayPct|regulationAndOtWins|seasonId|shotsAgainstPerGame|shotsForPerGame|teamFullName|teamId|ties|wins|winsInRegulation|winsInShootout|
+-------------+-----------+------------+-------------------+--------+---------------+------+--------+-----------------+--------------+--------+------+---------------+------------+-------------------+--------+-------------------+---------------+------------+------+----+----+----------------+--------------+
|         NULL|         80|         387|               NULL|     303|          

#### This request is a test run to view output of the request for Sample Team Chicago Blackhawks###

In [12]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, BooleanType

# Initialize Spark session
spark = SparkSession.builder.appName("FabricNotebook").getOrCreate()

# Requesting initial schema to understand structure/Test of team API before pull
url = "https://records.nhl.com/site/api/player/byTeam/16"
response = requests.get(url)
schemaskater = response.json()

# Extract the 'data' key
schemaskater_data = schemaskater.get('data', [])

# Convert to DataFrame if data exists
if schemaskater_data:
    # Infer schema using Spark for the given JSON
    df = spark.read.json(spark.sparkContext.parallelize(schemaskater_data))
    
    # Show schema of the DataFrame
    print("Schema of the dataset:")
    df.printSchema()
    
    # Show datatypes of each column
    print("Datatypes of each column:")
    for column, dtype in df.dtypes:
        print(f"Column: {column}, Type: {dtype}")

    # Display first few rows to verify the content
    df.show(truncate=False)
else:
    print("No data found in the response.")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 13, Finished, Available, Finished)

Schema of the dataset:
root
 |-- _corrupt_record: string (nullable = true)

Datatypes of each column:
Column: _corrupt_record, Type: string
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### ##RAW:DOWNLOAD ALL PLAYER IDS for a team

In [ ]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Assuming SparkSession has already been created
spark = SparkSession.builder.getOrCreate()

skater_list = []  # Temporary list to store all responses with TeamID

# Collect the TeamId column into a list
# Assuming unique_teams DataFrame has been created beforehand
team_ids = [row.TeamId for row in unique_teams.select("TeamId").collect()]

# Iterating through all TeamIds
for teamid in team_ids:
    teamdetails = requests.get(f"https://records.nhl.com/site/api/player/byTeam/{teamid}")
    
    if teamdetails.status_code == 200:  # Ensure the request was successful
        teams_data = teamdetails.json()  # Parse JSON from the response
        if "data" in teams_data:  # Assuming the actual data is nested under a key called 'data'
            for player in teams_data["data"]:
                player["teamId"] = teamid  # Add TeamID as a new field to each player dictionary
                skater_list.append(player)  # Append the updated player data to the list

# Creating a DataFrame from the collected list
if skater_list:  # Ensure the list is not empty before creating DataFrame
    # Infer schema manually to avoid PySparkValueError
    schema = StructType([
        StructField("id", StringType(), True),
        StructField("firstName", StringType(), True),
        StructField("lastName", StringType(), True),
        StructField("fullName", StringType(), True),
        StructField("teamId", IntegerType(), True),
        StructField("position", StringType(), True),
        StructField("accruedSeasons", IntegerType(), True),
        StructField("addNames", StringType(), True),
        StructField("ageSignWaiver", IntegerType(), True),
        StructField("ageSignelFa", IntegerType(), True),
        StructField("alert", StringType(), True),
        StructField("birthCity", StringType(), True),
        StructField("birthCountry", StringType(), True),
        StructField("birthDate", StringType(), True),
        StructField("birthStateProvince", StringType(), True),
        StructField("careerTeamId", IntegerType(), True),
        StructField("centralRegistryPosition", StringType(), True),
        StructField("clubElecArb", StringType(), True),
        StructField("currentTeamId", IntegerType(), True),
        StructField("dateOfDeath", StringType(), True),
        StructField("deceased", BooleanType(), True),
        StructField("epPlayerId", IntegerType(), True),
        StructField("faGroupAfterSeason", StringType(), True),
        StructField("firstSignedByTeamId", IntegerType(), True),
        StructField("freeAgentGroup", StringType(), True),
        StructField("group5Election", StringType(), True),
        StructField("group5SeasonsEarned", IntegerType(), True),
        StructField("group6Proration", StringType(), True),
        StructField("group6SeasonsEarned", IntegerType(), True),
        StructField("groupsEarnedThruSeason", IntegerType(), True),
        StructField("height", IntegerType(), True),
        StructField("hofInductionYear", StringType(), True),
        StructField("homeTown", StringType(), True),
        StructField("iihfHofInductionYear", StringType(), True),
        StructField("inHockeyHof", BooleanType(), True),
        StructField("inIihfHof", IntegerType(), True),
        StructField("inTop100Alltime", BooleanType(), True),
        StructField("inUsHockeyHof", BooleanType(), True),
        StructField("isDefected", StringType(), True),
        StructField("isDeleted", StringType(), True),
        StructField("isJunior", StringType(), True),
        StructField("isRetired", StringType(), True),
        StructField("isRookie", StringType(), True),
        StructField("isSuspended", StringType(), True),
        StructField("lastAmateurLeagueId", IntegerType(), True),
        StructField("lastAmateurTeamId", IntegerType(), True),
        StructField("lastNHLTeamId", IntegerType(), True),
        StructField("loanCapException", StringType(), True),
        StructField("longTermInjury", StringType(), True),
        StructField("message", StringType(), True),
        StructField("middleName", StringType(), True),
        StructField("nationality", StringType(), True),
        StructField("nhlExperience", IntegerType(), True),
        StructField("onRoster", StringType(), True),
        StructField("platformYear", IntegerType(), True),
        StructField("prName", StringType(), True),
        StructField("prStat", IntegerType(), True),
        StructField("proYearReduction", StringType(), True),
        StructField("reentryWaivers", StringType(), True),
        StructField("rosterSpecialCode", StringType(), True),
        StructField("salaryArbitrationExp", IntegerType(), True),
        StructField("shootsCatches", StringType(), True),
        StructField("sweaterNumber", IntegerType(), True),
        StructField("updateTimestamp", StringType(), True),
        StructField("usHofInductionYear", StringType(), True),
        StructField("vetCapExcptn", StringType(), True),
        StructField("waiverAmount", IntegerType(), True),
        StructField("waiverDraft", StringType(), True),
        StructField("waiverStatus", StringType(), True),
        StructField("weight", IntegerType(), True),
        StructField("yearsPro", IntegerType(), True),
       # StructField("teamId", IntegerType(), True),
        
        # Add other fields as needed based on the schema structure
    ])
    
    skater_df = spark.createDataFrame(skater_list, schema=schema)
    
    # Displaying the DataFrame (this assumes you are using a compatible environment like Databricks)
    skater_df = skater_df.withColumnRenamed("id", "PlayerID")

    skater_df.show()
    
    # Save the skater DataFrame to a CSV file
    skater_df.write.mode("overwrite").option("header", "true").csv("Files/NHL_RAW/FranchisePlayers.csv")
    
    print("Data successfully saved to CSV.")
else:
    print("No skater data available to create a DataFrame.")


StatementMeta(, , , Cancelled, , Cancelled)

A Test of Data Above

In [ ]:
df = spark.read.option("header", "true").csv("Files/NHL_RAW/FranchisePlayers.csv")
# df now is a Spark DataFrame containing JSON data from "Files/NHL_RAW/FranchisePlayers.json".
display(df)

StatementMeta(, , , Cancelled, , Cancelled)

## STANDINGS - RAW: Store team standings as JSON

## ##RAW: Fetch Franchise List and Store Data in JSON

In [14]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("NHL Franchise Data").getOrCreate()

# Fetch data from the API
summary = requests.get("https://records.nhl.com/site/api/franchise")
summary_franchise = summary.json()  # Parse JSON from the response

# Extract data from the response
summary_data = summary_franchise.get('data', [])

# Convert the data into a Pandas DataFrame
df_pd = pd.DataFrame(summary_data)

# Convert the Pandas DataFrame into a Spark DataFrame
df = spark.createDataFrame(df_pd)

# Select individual fields directly from the dataframe
df_final = df.select(
    col("firstSeasonId"),
    col("fullName"),
    col("id"),
    col("lastSeasonId"),
    col("mostRecentTeamId"),
    col("teamAbbrev"),
    col("teamCommonName"),
    col("teamPlaceName")
)


# Write the final DataFrame to JSON
df_final.write.mode("overwrite").json("Files/NHL_RAW/TeamsFirstLastSeason.json")
display(df_final)


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7e41ccd6-59b3-4a0b-a44e-13da04e6e32d)

# Run a function to clean FirstSeasonLastSeason and then change into CSV

In [17]:
# Code generated by Data Wrangler for pandas sample

def clean_data(pandas_df_final):
    # Select columns: 'firstSeasonId', 'fullName' and 6 other columns
    pandas_df_final = pandas_df_final.loc[:, ['firstSeasonId', 'fullName', 'id', 'lastSeasonId', 'mostRecentTeamId', 'teamAbbrev', 'teamCommonName', 'teamPlaceName']]
    return pandas_df_final

# Loaded variable 'df_final' from kernel state
pandas_df_final = df_final.toPandas()

pandas_df_final_clean = clean_data(pandas_df_final.copy())
pandas_df_final_clean.head()

StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 18, Finished, Available, Finished)

,firstSeasonId,fullName,id,lastSeasonId,mostRecentTeamId,teamAbbrev,teamCommonName,teamPlaceName
0,19171918,Montréal Canadiens,1,NaN,8,MTL,Canadiens,Montréal
1,19171918,Montreal Wanderers,2,19171918.0,41,MWN,Wanderers,Montreal
2,19171918,St. Louis Eagles,3,19341935.0,45,SLE,Eagles,St. Louis
3,19191920,Hamilton Tigers,4,19241925.0,37,HAM,Tigers,Hamilton
4,19171918,Toronto Maple Leafs,5,NaN,10,TOR,Maple Leafs,Toronto


In [18]:
# Code generated by Data Wrangler for PySpark DataFrame

def clean_data(df_final):
    # Select columns: 'firstSeasonId', 'fullName' and 6 other columns
    df_final = df_final.select('firstSeasonId', 'fullName', 'id', 'lastSeasonId', 'mostRecentTeamId', 'teamAbbrev', 'teamCommonName', 'teamPlaceName')
    return df_final

df_final_clean = clean_data(df_final)
display(df_final_clean)
#Test to see if it makes it
#Did you get to update
df_final_clean.write.option("header", "true").mode("overwrite").csv("Files/NHL_ETL/TEAMS_ETL/TeamsFirstLastSeason.csv")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e5fa243d-97ed-4547-93c8-8802e3b0142b)

#### <mark>##### BONUS CAN BE MOVED TO ANOTHER JOB
#### </mark>
### **RISK ** THIS ONLY RETURNS 135 records..which seems low for players all time.
**## Also beware of what of the fact for some reason changing it into Json is removing record count??**

##### **These are only leaders in Skating goals**
##### 

In [19]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType

# Start Spark session
spark = SparkSession.builder.appName("NHL Data Processing").getOrCreate()

# Fetching data from the API
response = requests.get("https://api.nhle.com/stats/rest/en/leaders/skaters/goals")
skater_goals = response.json()

# Extract data assuming the key is 'data'
skater_goals_data = skater_goals.get('data', [])

# Define the schema for nested JSON data
schema = StructType([
    StructField("goals", IntegerType()),
    StructField("player", StructType([
        StructField("id", IntegerType()),
        StructField("currentTeamId", StringType()),
        StructField("firstName", StringType()),
        StructField("lastName", StringType()),
        StructField("fullName", StringType()),
        StructField("positionCode", StringType()),
        StructField("sweaterNumber", StringType()),
    ])),
    StructField("team", StructType([
        StructField("id", IntegerType()),
        StructField("franchiseId", IntegerType()),
        StructField("fullName", StringType()),
        StructField("leagueId", IntegerType()),
        StructField("logos", ArrayType(StructType([
            StructField("id", IntegerType()),
            StructField("background", StringType()),
            StructField("endSeason", IntegerType()),
            StructField("secureUrl", StringType()),
            StructField("startSeason", IntegerType()),
            StructField("url", StringType())
        ])))
    ]))
])

# Create a DataFrame using the schema
df = spark.createDataFrame(skater_goals_data, schema=schema)

# Explode the logos array if you need to flatten this part as well
df = df.withColumn("exploded_logos", explode(col("team.logos"))).select(
    col("*"),
    col("exploded_logos.*")
).drop("logos", "exploded_logos")

display(df)
# Save the DataFrame as a JSON file
df.write.mode("overwrite").json("Files/NHL_RAW/skater_goals.json")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 20, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2bcc0af7-4ed2-437d-9cce-12f580c9305d)

## ```
## This is the Player Config and should only be accessed after the ETL stage due to the modifications of the nested summarries.
## 
### <mark>##### **##Dificulty Level Hard:**</mark>

In [20]:
config_players_request = requests.get("https://api.nhle.com/stats/rest/en/config")
config_players = config_players_request.json()

config_players = config_players.get('playerReportData',{})

config_players = spark.createDataFrame([config_players])
display(config_players)


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 21, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 383410b5-7f78-44fe-9afd-1251a78a11fc)

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Initialize Spark Session
spark = SparkSession.builder.appName("NHL Data Extraction").getOrCreate()
response = requests.get("https://api.nhle.com/stats/rest/en/leaders/skaters/assist")
skater_assist_data = response.json()
print(skater_assist_data)
# Extract data assuming the key is 'data'
skater_assist_data = skater_assist_data.get('data', [])
#print(skater_assist_data)
# Assuming 'skater_assist' is already fetched and loaded as shown in your code snippet
# Let's simulate fetching this into a DataFrame for the purpose of demonstration

# Example Schema Based on Your Data
schema = StructType([
    StructField("assists", IntegerType()),
    StructField("player", StructType([
        StructField("fullName", StringType()),
        StructField("sweaterNumber", StringType()),
        StructField("lastName", StringType()),
        StructField("id", StringType()),
        StructField("currentTeamId", StringType()),
        StructField("positionCode", StringType()),
    ])),
    StructField("team", StructType([
        StructField("id", IntegerType()),
        StructField("name", StringType()),
        # Add more fields if the 'team' dictionary has more fields
    ]))
])

# Create DataFrame from the data
df = spark.createDataFrame(skater_assist_data, schema)

# Selecting and flattening nested fields
df = df.select(
    col("assists"),
    col("player.fullName").alias("player_fullName"),
    col("player.sweaterNumber").alias("player_sweaterNumber"),
    col("player.lastName").alias("player_lastName"),
    col("player.id").alias("player_id"),
    col("player.currentTeamId").alias("player_currentTeamId"),
    col("player.positionCode").alias("player_positionCode"),
    col("team.id").alias("team_id"),
    col("team.name").alias("team_name")
    # Add more fields here if needed
)

# Show the DataFrame to verify the structure
df.show()

# Save the DataFrame if needed
df.write.mode("overwrite").json("Files/NHL_RAW/skater_assists.json")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 22, Finished, Available, Finished)

{'message': "CayenneRuntimeException Unable to resolve path:  (unknown 'assist' component)"}
+-------+---------------+--------------------+---------------+---------+--------------------+-------------------+-------+---------+
|assists|player_fullName|player_sweaterNumber|player_lastName|player_id|player_currentTeamId|player_positionCode|team_id|team_name|
+-------+---------------+--------------------+---------------+---------+--------------------+-------------------+-------+---------+
+-------+---------------+--------------------+---------------+---------+--------------------+-------------------+-------+---------+



In [22]:

df = spark.read.option("multiline", "true").json("Files/NHL_RAW/skater_assists.json")
# df now is a Spark DataFrame containing JSON data from "Files/NHL/skater_assists.json".
display(df)

StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1d644431-e20a-4dc9-833e-4ebde619864f)

## This Cell is temporarily frozed as it may just need to live on a dashbaord.

In [ ]:
df = spark.read.format("csv").option("header","true").load("Files/NHL_ETL/STANDINGS_ETL/teams_summary.csv")
# df now is a Spark DataFrame containing CSV data from "Files/NHL_ETL/STANDINGS_ETL/teams_summary.csv".
display(df)

StatementMeta(, , , Cancelled, , Cancelled)

## ##RAW: FETCH FRANCHISE CAPTAIN HISTORICAL RECORDS

In [23]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Assuming SparkSession has already been created
spark = SparkSession.builder.getOrCreate()
management_list = []  # Temporary list to store all responses with TeamID

# Collect the TeamId column into a list
# Assuming unique_teams DataFrame has been created beforehand
team_ids = [row.TeamId for row in unique_teams.select("TeamId").collect()]

# Iterating through all TeamIds
for teamid in team_ids:
    franchise_management_stats = requests.get(f"https://records.nhl.com/site/api/franchise-detail?cayenneExp=mostRecentTeamId={teamid}")
    
    if franchise_management_stats.status_code == 200:  # Ensure the request was successful
        teams_data = franchise_management_stats.json()  # Parse JSON from the response
        if "data" in teams_data:  # Assuming the actual data is nested under a key called 'data'
            for record in teams_data["data"]:
                record["teamId"] = teamid  # Add TeamID as a new field to each player dictionary
                management_list.append(record)  # Append the updated goalie data to the list

display(management_list)
management_df = spark.createDataFrame(management_list)

management_df.write.mode("overwrite").option("header", "true").json("Files/NHL_RAW/Franchise_Coach_Captain_hisotry.json")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 24, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 56286c5a-5f0d-4bf0-a06e-b0dca2785c03)

# ###PREP: Getting all CLUB Schedules for all season games

In [24]:
#create list for Hockey_Seasons - 
# First recorded hockey season
start_year = 1917
# Current hockey season end year
end_year = 2025

# Generate the list of hockey seasons
hockey_seasons = [f"{year}{year+1}" for year in range(start_year, end_year)]

# Print the list
print(hockey_seasons)


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 25, Finished, Available, Finished)

['19171918', '19181919', '19191920', '19201921', '19211922', '19221923', '19231924', '19241925', '19251926', '19261927', '19271928', '19281929', '19291930', '19301931', '19311932', '19321933', '19331934', '19341935', '19351936', '19361937', '19371938', '19381939', '19391940', '19401941', '19411942', '19421943', '19431944', '19441945', '19451946', '19461947', '19471948', '19481949', '19491950', '19501951', '19511952', '19521953', '19531954', '19541955', '19551956', '19561957', '19571958', '19581959', '19591960', '19601961', '19611962', '19621963', '19631964', '19641965', '19651966', '19661967', '19671968', '19681969', '19691970', '19701971', '19711972', '19721973', '19731974', '19741975', '19751976', '19761977', '19771978', '19781979', '19791980', '19801981', '19811982', '19821983', '19831984', '19841985', '19851986', '19861987', '19871988', '19881989', '19891990', '19901991', '19911992', '19921993', '19931994', '19941995', '19951996', '19961997', '19971998', '19981999', '19992000', '20

## RAW" Test Version Below Pull and store just one season of Franchise Schedule
###This is just the blackhawks though ###

In [25]:
import requests
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, ArrayType, MapType

# Initialize Spark session
spark = SparkSession.builder.appName("Franchise Season Schedule").getOrCreate()
datalake_output_path = "Files/NHL_RAW/Franchise_Seasons_Schedules.json"

FranchiseName = "CHI"
Season = "20242025"

# Define schema for the JSON data
game_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", IntegerType(), True),
    StructField("gameType", IntegerType(), True),
    StructField("gameDate", StringType(), True),
    StructField("venue", StructType([
        StructField("default", StringType(), True)
    ])),
    StructField("neutralSite", BooleanType(), True),
    StructField("startTimeUTC", StringType(), True),
    StructField("awayTeam", StructType([
        StructField("id", IntegerType(), True),
        StructField("commonName", MapType(StringType(), StringType()), True),
        StructField("placeName", MapType(StringType(), StringType()), True),
        StructField("abbrev", StringType(), True),
        StructField("score", IntegerType(), True)
    ])),
    StructField("homeTeam", StructType([
        StructField("id", IntegerType(), True),
        StructField("commonName", MapType(StringType(), StringType()), True),
        StructField("placeName", MapType(StringType(), StringType()), True),
        StructField("abbrev", StringType(), True),
        StructField("score", IntegerType(), True)
    ]))
])

# Fetch data from the API
url = (f"https://api-web.nhle.com/v1/club-schedule-season/{FranchiseName}/{Season}")
response = requests.get(url)
response.raise_for_status()  # Raise an error if the request failed

# Parse the JSON response
data = response.json()

data_string = json.dumps(data)  # Convert the JSON to a string

# Load the JSON string into a PySpark DataFrame
raw_data = spark.read.json(spark.sparkContext.parallelize([data_string]), schema=StructType([
    StructField("games", ArrayType(game_schema), True)
]))

# Flatten the games array into a DataFrame
games_df = raw_data.selectExpr("explode(games) as game").select("game.*")
games_df = games_df.withColumnRenamed("id", "Season_Game_ID")

# Display the DataFrame schema for verification
display(games_df)

# Show a preview of the DataFrame
games_df.show(truncate=False)
games_df.write.mode("overwrite").json(datalake_output_path)

# Save the DataFrame to a Delta table or another format as needed
#games_df.write.format("delta").save("/path/to/output")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3ff8699d-a42c-474b-a0a9-bd9b5e11a0fb)

+--------------+--------+--------+----------+--------------------------+-----------+--------------------+------------------------------------------------------------+----------------------------------------------------------------------------------------------+
|Season_Game_ID|season  |gameType|gameDate  |venue                     |neutralSite|startTimeUTC        |awayTeam                                                    |homeTeam                                                                                      |
+--------------+--------+--------+----------+--------------------------+-----------+--------------------+------------------------------------------------------------+----------------------------------------------------------------------------------------------+
|2024010031    |20242025|1       |2024-09-25|{United Center}           |false      |2024-09-26T00:00:00Z|{17, {default -> Red Wings}, {default -> Detroit}, DET, 4}  |{16, {default -> Blackhawks}, {default -> Chicag

### RAW STAGING: TEAM SEASONS Below Version WORKS NOW......almost am having trouble appending to a dictionary !

In [30]:
df = spark.read.option("multiline", "true").json("Files/NHL_RAW/Franchise_Seasons_Schedules.json")
# df now is a Spark DataFrame containing JSON data from "Files/NHL_RAW/Franchise_Seasons_Schedules.json".
display(df)
display(games_df)

StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c03dcb23-0d50-4e3a-841b-cdd8ae207934)

SynapseWidget(Synapse.DataFrame, 3fe71d43-5636-48ac-bf00-18c499abc3b5)

# Test Version of loop breakdown
## Used to figure out why data does not sore in variant column for json.

In [26]:
##Lock temporarily, use clone below for torubleshooting loop

import requests
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import json

# File paths
teams_file_path = "Files/NHL_ETL/TEAMS_ETL/TeamsFirstLastSeason.csv"
datalake_output_path = "Files/NHL_RAW/Franchise_Seasons_Schedules.json"

# Load the teams data
df_teams = spark.read.format("csv").option("header", "true").load(teams_file_path)

# Initialize an empty list to collect all the data
teams_seasons_schedule = []

# Collect team abbreviations and season IDs
team_abbrevs = df_teams.select("teamAbbrev", "firstSeasonId", "lastSeasonId").collect()
#print(team_abbrevs)
# Iterate through each team and season to fetch data
for row in team_abbrevs:
    team_abbrev = row["teamAbbrev"]

    for season in hockey_seasons:
        season = int(season)

        # Fetch data from the API
        url = f"https://api-web.nhle.com/v1/club-schedule-season/{team_abbrev}/{season}"
        
        response = requests.get(url)
        url_status = response.status_code
       # print(response)
        #print(type(response))
       # print(f"{team_abbrev}{season}")
        if url_status == 200:
            season_data = response.json()
            print(f"{team_abbrev}{season} storing data ")
            season_data_with_context = {
                "teamAbbrev": team_abbrev,
                "season": season,
                "data": season_data.get('data', [])  # Store as JSON string
            }
            teams_seasons_schedule.append(season_data_with_context)

        elif url_status == 404:
            print(f"{team_abbrev}{season} data failed to be loaded due to {url_status} ")


            # Append the data to the list
        else:
            print("nothing to return here.")

print(teams_seasons_schedule)



StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 27, Finished, Cancelled, Cancelled)

MTL19171918 storing data 
MTL19181919 storing data 
MTL19191920 storing data 
MTL19201921 storing data 
MTL19211922 storing data 
MTL19221923 storing data 
MTL19231924 storing data 
MTL19241925 storing data 
MTL19251926 storing data 
MTL19261927 storing data 
MTL19271928 storing data 
MTL19281929 storing data 
MTL19291930 storing data 
MTL19301931 storing data 
MTL19311932 storing data 
MTL19321933 storing data 
MTL19331934 storing data 
MTL19341935 storing data 
MTL19351936 storing data 
MTL19361937 storing data 
MTL19371938 storing data 
MTL19381939 storing data 
MTL19391940 storing data 
MTL19401941 storing data 
MTL19411942 storing data 
MTL19421943 storing data 
MTL19431944 storing data 
MTL19441945 storing data 
MTL19451946 storing data 
MTL19461947 storing data 
MTL19471948 storing data 
MTL19481949 storing data 
MTL19491950 storing data 
MTL19501951 storing data 
MTL19511952 storing data 
MTL19521953 storing data 
MTL19531954 storing data 
MTL19541955 storing data 
MTL19551956 

## Correction of Franchise SeasonLoop

In [27]:
import requests
import json
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, BooleanType, ArrayType, MapType
from pyspark.sql.functions import lit
import os

# File paths
teams_file_path = "Files/NHL_ETL/TEAMS_ETL/TeamsFirstLastSeason.csv"
datalake_output_path = "Files/NHL_RAW/Franchise_Seasons_Schedules.csv"

# Clear output path before writing
output_folder_path = datalake_output_path.replace(".csv", "")  # Ensure the folder is targeted
if os.path.exists(output_folder_path):
    shutil.rmtree(output_folder_path)
    print(f"Cleared the output folder: {output_folder_path}")

# Load the teams data
df_teams = spark.read.format("csv").option("header", "true").load(teams_file_path)

# Temporary testing data
hockey_seasons

# Initialize an empty list to collect all the DataFrames
teams_seasons_schedule = []

game_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("season", IntegerType(), True),
    StructField("gameType", IntegerType(), True),
    StructField("gameDate", StringType(), True),
    StructField("venue", StructType([
        StructField("default", StringType(), True)
    ])),
    StructField("neutralSite", BooleanType(), True),
    StructField("startTimeUTC", StringType(), True),
    StructField("awayTeam", StructType([
        StructField("id", IntegerType(), True),
        StructField("commonName", MapType(StringType(), StringType()), True),
        StructField("placeName", MapType(StringType(), StringType()), True),
        StructField("abbrev", StringType(), True),
        StructField("score", IntegerType(), True)
    ])),
    StructField("homeTeam", StructType([
        StructField("id", IntegerType(), True),
        StructField("commonName", MapType(StringType(), StringType()), True),
        StructField("placeName", MapType(StringType(), StringType()), True),
        StructField("abbrev", StringType(), True),
        StructField("score", IntegerType(), True)
    ]))
])

# Collect team abbreviations and season IDs
team_abbrevs = df_teams.select("teamAbbrev", "firstSeasonId", "lastSeasonId").collect()

# Iterate through each team and season to fetch data
for row in team_abbrevs:
    team_abbrev = row["teamAbbrev"]

    for season in hockey_seasons:
        season = int(season)

        # Fetch data from the API
        url = f"https://api-web.nhle.com/v1/club-schedule-season/{team_abbrev}/{season}"
        response = requests.get(url)
        url_status = response.status_code

        if url_status == 200:
            season_data = response.json()
            data_string = json.dumps(season_data)  # Convert the JSON to a string

            # Load the JSON string into a PySpark DataFrame
            raw_data = spark.read.json(spark.sparkContext.parallelize([data_string]), schema=StructType([
                StructField("games", ArrayType(game_schema), True)
            ]))

            # Flatten the games array into a DataFrame
            games_df = raw_data.selectExpr("explode(games) as game").select("game.*")
            games_df = games_df.withColumnRenamed("id", "Season_Game_ID")
            games_df = games_df.withColumn("Priority_Franchise_Schedule", lit(team_abbrev))

            # Collect all DataFrames
            teams_seasons_schedule.append(games_df)

        elif url_status == 404:
            print(f"{team_abbrev}{season} data failed to be loaded due to {url_status}")
        else:
            print("Nothing to return here.")

# Combine all DataFrames into one
if teams_seasons_schedule:
    teams_seasons_schedule_df = teams_seasons_schedule[0]
    for df in teams_seasons_schedule[1:]:
        teams_seasons_schedule_df = teams_seasons_schedule_df.union(df)

    # Save the final DataFrame as CSV
    teams_seasons_schedule_df.write.mode("overwrite").csv(output_folder_path, header=True)
    print(f"Data saved successfully to: {output_folder_path}")
else:
    print("No data to save.")


StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 28, Finished, Available, Finished)

MTL20042005 data failed to be loaded due to 404
MWN20042005 data failed to be loaded due to 404
SLE20042005 data failed to be loaded due to 404
HAM20042005 data failed to be loaded due to 404
TOR20042005 data failed to be loaded due to 404
BOS20042005 data failed to be loaded due to 404
MMR20042005 data failed to be loaded due to 404
BRK20042005 data failed to be loaded due to 404
QUA20042005 data failed to be loaded due to 404
NYR20042005 data failed to be loaded due to 404
CAR20042005 data failed to be loaded due to 404
COL20042005 data failed to be loaded due to 404
ARI20042005 data failed to be loaded due to 404
SJS20042005 data failed to be loaded due to 404
OTT20042005 data failed to be loaded due to 404
CHI20042005 data failed to be loaded due to 404
DET20042005 data failed to be loaded due to 404
CLE20042005 data failed to be loaded due to 404
LAK20042005 data failed to be loaded due to 404
DAL20042005 data failed to be loaded due to 404
CBJ20042005 data failed to be loaded due

AnalysisException: [UNSUPPORTED_DATA_TYPE_FOR_DATASOURCE] The CSV datasource doesn't support the column `venue` of the type "STRUCT<default: STRING>".

In [28]:

df = spark.read.option("multiline", "true").json("Files/NHL_RAW/Franchise_Seasons_Schedules.json")
# df now is a Spark DataFrame containing JSON data from "Files/NHL_RAW/Franchise_Seasons_Schedules.json".
display(df)

StatementMeta(, ef783bc8-eecd-48ca-8563-1023d25de0cb, 29, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, edeae9d9-59f7-4b91-94f5-cd2cc457ad4e)

## Original V1 Run of Fancchise Seasons

In [ ]:
##Lock temporarily, use clone above for torubleshooting loop
##This is the original...V12

import requests
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import json

# File paths
teams_file_path = "Files/NHL_ETL/TEAMS_ETL/TeamsFirstLastSeason.csv"
datalake_output_path = "Files/NHL_RAW/Franchise_Seasons_Schedules.json"

# Load the teams data
df_teams = spark.read.format("csv").option("header", "true").load(teams_file_path)

# Initialize an empty list to collect all the data
teams_seasons_schedule = []

# Collect team abbreviations and season IDs
team_abbrevs = df_teams.select("teamAbbrev", "firstSeasonId", "lastSeasonId").collect()
print(team_abbrevs)
# Iterate through each team and season to fetch data
for row in team_abbrevs:
    team_abbrev = row["teamAbbrev"]

    for season in hockey_seasons:
        season = int(season)

        # Fetch data from the API
        url = f"https://api-web.nhle.com/v1/club-schedule-season/{team_abbrev}/{season}"
        response = requests.get(url)

        # Parse the JSON response
        try:
            season_data = response.json()
          #  print(season_data) #a test of url pull
            print("\n ")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for team {team_abbrev}, season {season}: {e}")
            season_data = ""

        # Add the team abbreviation, season ID, and raw JSON string
        season_data_with_context = {
            "teamAbbrev": team_abbrev,
            "season": season,
            "data": json.dumps(season_data.get('data', {}))  # Store as JSON string
        }
       # print(season_data_with_context)

        # Append the data to the list
        teams_seasons_schedule.append(season_data_with_context)

# Define the schema to match the data structure
schema = StructType([
    StructField("teamAbbrev", StringType(), True),
    StructField("season", IntegerType(), True),
    StructField("data", StringType(), True)  # Store `data` as a JSON string
])

# Convert the Python list to a Spark DataFrame
df_schedule = spark.createDataFrame(teams_seasons_schedule, schema=schema)

# Save the DataFrame as JSON to the specified path
#df_schedule.write.mode("overwrite").json(datalake_output_path)

# Display the DataFrame
display(df_schedule)


StatementMeta(, , , Cancelled, , Cancelled)

In [ ]:
df = spark.read.option("multiline", "true").json("Files/NHL_RAW/Franchise_Seasons_Schedules.json")
# df now is a Spark DataFrame containing JSON data from "Files/NHL_RAW/Franchise_Seasons_Schedules.json".
display(df)

StatementMeta(, , , Cancelled, , Cancelled)

# Download BoxScore

Returns far less detail than feed/live | and is much more suitable for post-game details including goals, shots, PIMs, blocked, takeaways, giveaways and hits.


In [ ]:
import request 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from datetime import datetime

today = datetime.today()
date_today = today().date


df = spark.read.option("multiline", "true").json("Files/NHL_RAW/Franchise_Seasons_Schedules.json")
# df now is a Spark DataFrame containing JSON data from "Files/NHL_RAW/Franchise_Seasons_Schedules.json".
display(df)

gameid_list = df_teams.select("Season_Game_Id", "game_date").collect()
boxscore = []

for game in gameid_list["game_date"] <= date_today:
    gameid = row["Season_Game_Id"]

    url = (f"https://statsapi.web.nhl.com/api/v1/game/{gameid}/boxscore")
    response = requests.get(url)
    url_status = response.status_code
    if url_status == 200:
        boxscore_response = response.json()
        print(boxscore_response)
        boxscore.append(boxscore_response)
        #data_string = json.dumps(season_data)  # Convert the JSON to a string
    elif url_status == 400:
        print(f"Could not review boxscore for the following games {gameid}")


    
boxscore.write.mode.("Overwrite").json("Files/NHL_RAW/RAW_GameBox.json")



StatementMeta(, , , Cancelled, , Cancelled)

## ##RAW GET GOALIE DETIALS for each team
# 

In [ ]:
##Lock temporarily, use clone above for torubleshooting loop

import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Assuming SparkSession has already been created
spark = SparkSession.builder.getOrCreate()
goalie_list = []  # Temporary list to store all responses with TeamID

# Collect the TeamId column into a list
# Assuming unique_teams DataFrame has been created beforehand
team_ids = [row.TeamId for row in unique_teams.select("TeamId").collect()]
goalie_stats_schema = requests.get(f"https://records.nhl.com/site/api/franchise-goalie-records?cayenneExp=franchiseId=16")
display(goalie_stats_schema)
# Iterating through all TeamIds
for teamid in team_ids:
    franchise_goalie_stats = requests.get(f"https://records.nhl.com/site/api/franchise-goalie-records?cayenneExp=franchiseId={teamid}")
    
    if franchise_goalie_stats.status_code == 200:  # Ensure the request was successful
        teams_data = franchise_goalie_stats.json()  # Parse JSON from the response
        if "data" in teams_data:  # Assuming the actual data is nested under a key called 'data'
            for record in teams_data["data"]:
                record["teamId"] = teamid  # Add TeamID as a new field to each player dictionary
                goalie_list.append(record)  # Append the updated goalie data to the list

display(goalie_list)
goalie_df = spark.createDataFrame(goalie_list)

goalie_df.write.mode("overwrite").option("header", "true").json("Files/NHL_RAW/Franchise_Goalie_History.json")




StatementMeta(, , , Cancelled, , Cancelled)